<a href="https://colab.research.google.com/github/sverdoot/DL-in-NLP-course/blob/master/workshop%201/task2_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2vec preprocessing

Preprocessing is not the most interesting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/textdata). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parced from the internet)
1. tokenization
1. building the vocabulary and choosing its size
1. assigning each token a number (numericalization)
1. data structuring and batching

Your goal is to make SkipGramBatcher class which returns two numpy tensors with word indices. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpfull to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `(batch_size, 2*window_size)`, `(batch_size,)` for CBOW or `(batch_size,)`, `(batch_size,)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(batch_size, window_size, ...), SkipGram(num_skips, skip_window). You should implement only one batcher in this task, it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in separate file. It will be reused for the next task. Result of your work should represent that your batch have proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
bag_window = 2

batch = [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

labels = ['against', 'early', 'working', 'class']
```

If you struggle with somethng, ask your neighbour. If it is not obvious for you, probably someone else is looking for the answer too. And in contrast, if you see that you can help someone - just do it! Good luck!

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)
import numpy as np

import collections, math, os, random, zipfile
import numpy as np

from six.moves import urllib
from six.moves import xrange

url = 'http://mattmahoney.net/dc/'

UNK_TOKEN = '<UNK>'

import requests, os
from os.path import isfile

In [0]:
if not os.path.isfile('text8'):
    with open('data.zip', 'wb') as f:
        r = requests.get('http://mattmahoney.net/dc/text8.zip')
        f.write(r.content)
    !unzip 'data.zip' 

def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        data = (f.read(f.namelist()[0]).split())
        data = list(map(lambda x: x.decode(), data))
    return data

!ls
#words = read_data('text8')
with open('text8') as f:
    words = f.read().split()

Archive:  data.zip
  inflating: text8                   
data.zip  sample_data  text8


In [0]:
VOCAB_SIZE = 10000
full_data = words
freq_dict = collections.Counter(full_data)
no_rare_dict = freq_dict.most_common(VOCAB_SIZE)
min_freq = no_rare_dict[-1][1]
vocabulary = [x[0] for x in no_rare_dict]
vocabulary.append(UNK_TOKEN)

data = []
for i, word in enumerate(full_data):
  if freq_dict[word] > min_freq:
    data.append(word)
  else:
    data.append(UNK_TOKEN)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

data_idx = 0

class Batcher:
  
  def __init__(self, batch_size, window_size, data, data_idx=0):
    self.batch_size = batch_size
    self.window_size = window_size
    self.data = data
    
  def __iter__(self):
  
    return self

  def __next__(self, data_idx_=None):
    window_size = self.window_size
    batch_size = self.batch_size
    data = self.data
    global data_idx
    batch = []
    labels = [] 
  
    context = collections.deque(maxlen=2 * window_size + 1)
  
    for _ in range(2 * window_size + 1):
      context.append(word2idx[data[data_idx]])
      data_idx = (data_idx + 1) % len(data)
  
    for i in range(batch_size):
      batch.append([context[j] for  j in list(range(window_size)) + list(range(window_size + 1, 2 * window_size + 1))])
      labels.append(context[window_size])
      context.append(word2idx[data[data_idx]])
      data_idx = (data_idx + 1) % len(data)
    
    return (batch, labels)

In [0]:
batch_size = 16
window_size = 2
batcher = Batcher(batch_size, window_size, data)
build_batch = iter(batcher)

In [0]:
print('data:', [di for di in data[:16]])

data_index = 0
batch, labels = next(build_batch)
print('\nwith window_size = %d:' %2)
print('    batch:', [[idx2word[b] for b in bi] for bi in batch])
print('    labels:', [idx2word[li] for li in labels])
data: ['', 'a', 'the', 'cat', 'and', 'a', 'the', 'the']

data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', '<UNK>', 'including', 'the']

with window_size = 2:
    batch: [['anarchism', 'originated', 'a', 'term'], ['originated', 'as', 'term', 'of'], ['as', 'a', 'of', 'abuse'], ['a', 'term', 'abuse', 'first'], ['term', 'of', 'first', 'used'], ['of', 'abuse', 'used', 'against'], ['abuse', 'first', 'against', 'early'], ['first', 'used', 'early', 'working'], ['used', 'against', 'working', 'class'], ['against', 'early', 'class', '<UNK>'], ['early', 'working', '<UNK>', 'including'], ['working', 'class', 'including', 'the'], ['class', '<UNK>', 'the', '<UNK>'], ['<UNK>', 'including', '<UNK>', 'of'], ['including', 'the', 'of', 'the'], ['the', '<UNK>', 'the', 'english']]
    labels: ['as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', '<UNK>', 'including', 'the', '<UNK>', 'of']
